In [1]:
import numpy as np
import arcpy
from arcpy import sa
from arcpy.sa import *

In [13]:
## Построение карт квантилей 5% и 95% 
arcpy.CheckOutExtension("Spatial")

nee_layer = arcpy.Raster("E:/GCAS/gcas_bio_opt_2010_2019/gcas_bio_opt_2019.tif")
mask_raster = arcpy.Raster("E:/ecoregions_palearctic_steppe/ecoregions.tif")
arcpy.env.cellSize = "MINOF"
nee_layer = sa.ExtractByMask(nee_layer, mask_raster)
nee_array = arcpy.RasterToNumPyArray(nee_layer)

# Исключаем NoData
no_data_value = nee_layer.noDataValue
print(f"Значение NoData: {no_data_value}")

if no_data_value is not None and np.isfinite(no_data_value):
    mask = (nee_array != no_data_value)
else:
    mask = np.ones_like(nee_array, dtype=bool)
   
nee_array_masked = np.where(mask, nee_array, np.nan)
print(f"Число элементов после применения маски: {np.count_nonzero(~np.isnan(nee_array_masked))}")

if np.isnan(np.nanmin(nee_array_masked)):
    raise ValueError("Массив после маски состоит только из NaN значений.")

min_value = np.nanmin(nee_array_masked)
max_value = np.nanmax(nee_array_masked)
print(f"Минимальное значение: {min_value}, Максимальное значение: {max_value}")
q5 = np.nanpercentile(nee_array_masked, 5)
q95 = np.nanpercentile(nee_array_masked, 95)
print(f"5-й квантиль: {q5}, 95-й квантиль: {q95}")

if q5 >= q95:
    raise ValueError(f"5-й квантиль больше или равен 95-му: {q5} >= {q95}")

remap = RemapRange([
    [float(min_value), float(q5 + 1e-6), 1], # Включаем ровно 5% в 1-й класс
    [float(q5 + 1e-6), float(q95), 2],       # Второй класс (между 5-м и 95-м квантилями)
    [float(q95), float(max_value), 3]        # Включаем ровно 95% в 3-й класс
])

quantile_nee2019 = Reclassify(nee_layer, "VALUE", remap, "NODATA")
final_raster_path = "E:/GCAS/nee_quantile/quantile_nee2019.tif"
quantile_nee2019.save(final_raster_path)

arcpy.CheckInExtension("Spatial")
arcpy.Delete_management(nee_layer)

Значение NoData: None
Число элементов после применения маски: 269931375
Минимальное значение: -250.46934509277344, Максимальное значение: 149.02313232421875
5-й квантиль: -53.527164459228516, 95-й квантиль: 6.696890830993652


<Result 'true'>